In [431]:
## import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from itertools import groupby
warnings.filterwarnings('ignore')

In [432]:
#reading the data
loan_data = pd.read_csv("loan.csv")

In [433]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [434]:
loan_data.shape

(39717, 111)

## First identiable observation, is that lots of columns has nullified value
- Those columns will be dropped.
- Example is all fields related to co-borrowers; across the sheet, there's zero co-borrower, meaning that any column with "joint" can be safely dropped

In [435]:
loan_data.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [436]:
loan_data.dropna(axis = 1, how = 'all', inplace = True)
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,Sep-13,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,Apr-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,Jun-16,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0


## Also, there are others with single value (zero, 1, etc.)  for the whole column
- Meaning that they have no impact on the data
- Other columns not contributing to the study should be removed, such as "id", "member_id", "url", "title", "emp_title", "zip_code", "last_credit_pull_d", "addr_state", and others
- 'funded_amnt_inv' is always equals to 'funded_amnt', so it's considered as a duplicate

In [437]:
loan_data.columns[loan_data.nunique() == 1]

Index(['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens'],
      dtype='object')

In [438]:
loan_data.drop(loan_data.columns[loan_data.nunique() == 1], axis = 1, inplace = True)

In [439]:
# Dropping other data columns that include irrelevant information to the analysis
loan_data.drop(['id', 'member_id', 'url', 'title', 'emp_title', 'zip_code', 'last_credit_pull_d', 'addr_state','desc','out_prncp_inv','total_pymnt_inv', 'delinq_2yrs', 'revol_bal', 'out_prncp', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d' , 'mths_since_last_delinq', 'mths_since_last_record','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','total_acc','funded_amnt_inv'], axis = 1, inplace = True)

In [440]:
loan_data.shape

(39717, 17)

## Deriving new data field from existing
- I noticed some loans were funded with an amount less than the requested, so I'll double check if this behaviour is associated with defaulting.
- Also, it'd look cleaner if the issue_d was splited into month and year

In [441]:
# Defining new column with values 0 and 1 to reflect if the funded amount is less than the requested
loan_data['fully_funded'] = np.where(loan_data.columns[1] == loan_data.columns[0],0,1)

In [442]:
# Defining two new columns for year and month of issue_d
loan_data['issue_year']=pd.to_datetime(loan_data.issue_d,format='%b-%y').dt.year
loan_data['issue_month']=pd.to_datetime(loan_data.issue_d,format='%b-%y').dt.month

In [443]:
print('Number of columns:',loan_data.shape[1])
print('Number of rows:', loan_data.shape[0])

Number of columns: 20
Number of rows: 39717


### Now we have 20 columns:
- I'm going to analyze those fields to understand the driving factors of loan defaulting before approving loan.


### List of columns that might have impact on defaulting

In [444]:
loan_data.columns

Index(['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'purpose', 'dti',
       'revol_util', 'pub_rec_bankruptcies', 'fully_funded', 'issue_year',
       'issue_month'],
      dtype='object')

##### Fully paid and charged-off loans are to be considered in the analysis, so we need to eliminate loans with "Current" status

In [445]:
# Checking available Loan Status types
loan_data.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current'], dtype=object)

In [446]:
# Removing Current
loan_data = loan_data[loan_data.loan_status != "Current"]
loan_data.loan_status.unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

In [447]:
# Re-checking for columns with same value again.
loan_data.columns[loan_data.nunique() == 1]

Index(['fully_funded'], dtype='object')

In [448]:
# Drop them, to make sure of full cleansing
loan_data.drop(loan_data.columns[loan_data.nunique() == 1], axis = 1, inplace = True)

In [449]:
print('Number of columns:',loan_data.shape[1])
print('Number of rows:', loan_data.shape[0])
loan_data.columns

Number of columns: 19
Number of rows: 38577


Index(['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'purpose', 'dti',
       'revol_util', 'pub_rec_bankruptcies', 'issue_year', 'issue_month'],
      dtype='object')

##### All the remaining loans are in the same year of 2011, which is considered in itself one of the conclusions. I still need to dig deeper to know the reasons behind that observation.

### Checking if some columns still contain missing values

In [454]:
(loan_data.isna().sum()/len(loan_data.index))*100

loan_amnt               0.000000
funded_amnt             0.000000
term                    0.000000
int_rate                0.000000
installment             0.000000
grade                   0.000000
sub_grade               0.000000
emp_length              2.677761
home_ownership          0.000000
annual_inc              0.000000
verification_status     0.000000
issue_d                 0.000000
loan_status             0.000000
purpose                 0.000000
dti                     0.000000
revol_util              0.129611
pub_rec_bankruptcies    1.806776
issue_year              0.000000
issue_month             0.000000
dtype: float64

## Handling Missing values
 - There's only one column with missing values, which is "emp_length"
 - I need to check the content before taking decision on this field. 

In [274]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 4 to 18215
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   loan_amnt             1140 non-null   int64  
 1   funded_amnt           1140 non-null   int64  
 2   int_rate              1140 non-null   object 
 3   installment           1140 non-null   float64
 4   grade                 1140 non-null   object 
 5   sub_grade             1140 non-null   object 
 6   emp_length            1098 non-null   object 
 7   home_ownership        1140 non-null   object 
 8   annual_inc            1140 non-null   float64
 9   verification_status   1140 non-null   object 
 10  issue_d               1140 non-null   object 
 11  purpose               1140 non-null   object 
 12  dti                   1140 non-null   float64
 13  revol_util            1140 non-null   object 
 14  pub_rec_bankruptcies  1140 non-null   float64
dtypes: float64(4), int64

In [290]:
# Sort in descending order the counts of loans against the emp_length
print(loan_data.emp_length.value_counts().sort_values(ascending=False))
print("Number of NULL cells:", loan_data.emp_length.isnull().sum(), "which represents", round(loan_data.emp_length.isnull().sum()/len(loan_data.emp_length.index)*100,2), "%")
print("Number of NA cells:", loan_data.emp_length.isna().sum(), "which represents", round(loan_data.emp_length.isna().sum()/len(loan_data.emp_length.index)*100,2), "%")

10+ years    391
2 years       97
4 years       94
5 years       88
3 years       83
< 1 year      75
1 year        71
7 years       62
6 years       61
8 years       44
9 years       32
Name: emp_length, dtype: int64
Number of NULL cells: 42 which represents 3.68 %
Number of NA cells: 42 which represents 3.68 %


##### The above result shows that the number of employess with 10+ years is much higher than that of the next most frequent value. This means that we can safely assign that value to the null values in the column. Also the missing values are in very low percentage. So replacing with same value doesnt affect the analysis much.

In [296]:
loan_data.emp_length.fillna(loan_data.emp_length.mode()[0], inplace = True)
print(loan_data.emp_length.isna().sum())
print(loan_data.emp_length.isnull().sum())

0
0


In [297]:
loan_data.head()

,loan_amnt,funded_amnt,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,revol_util,pub_rec_bankruptcies
4,3000,3000,12.69%,67.79,B,B5,1 year,RENT,80000.0,Source Verified,Dec-11,other,17.94,53.90%,0.0
32,10000,10000,15.96%,242.97,C,C5,2 years,RENT,29120.0,Verified,Dec-11,debt_consolidation,22.83,63.60%,0.0
39,12500,12500,12.69%,282.44,B,B5,1 year,RENT,27000.0,Verified,Dec-11,debt_consolidation,16.04,50%,0.0
86,14000,14000,17.27%,349.98,D,D3,4 years,RENT,28000.0,Verified,Dec-11,other,6.34,79.40%,0.0
95,15300,15300,22.06%,423.10,F,F4,6 years,RENT,85000.0,Verified,Dec-11,credit_card,20.58,89.70%,0.0


## Analyzing remaining fields for further cleansing
- Understand the relation between 'int_rate' and 'grade' with the emp_length and bankrupcy

In [450]:
# Checking if the date/year can be utilized as a subset to split data
loan_data.issue_d.unique()

array(['Dec-11', 'Nov-11', 'Oct-11', 'Sep-11', 'Aug-11', 'Jul-11',
       'Jun-11', 'May-11', 'Apr-11', 'Mar-11', 'Feb-11', 'Jan-11',
       'Dec-10', 'Nov-10', 'Oct-10', 'Sep-10', 'Aug-10', 'Jul-10',
       'Jun-10', 'May-10', 'Apr-10', 'Mar-10', 'Feb-10', 'Jan-10',
       'Dec-09', 'Nov-09', 'Oct-09', 'Sep-09', 'Aug-09', 'Jul-09',
       'Jun-09', 'May-09', 'Apr-09', 'Mar-09', 'Feb-09', 'Jan-09',
       'Dec-08', 'Nov-08', 'Oct-08', 'Sep-08', 'Aug-08', 'Jul-08',
       'Jun-08', 'May-08', 'Apr-08', 'Mar-08', 'Feb-08', 'Jan-08',
       'Dec-07', 'Nov-07', 'Oct-07', 'Sep-07', 'Aug-07', 'Jul-07',
       'Jun-07'], dtype=object)

In [451]:
loan_data.issue_year.value_counts().sort_values()

2007      251
2008     1562
2009     4716
2010    11532
2011    20516
Name: issue_year, dtype: int64

In [453]:
loan_data.pub_rec_bankruptcies.unique()

array([ 0.,  1.,  2., nan])

In [423]:
# Sorting loan grades in relation to the int_rate
loan_data.grade.value_counts().sort_values(ascending=True)

G      299
F      976
E     2663
D     5085
C     7834
A    10045
B    11675
Name: grade, dtype: int64

## Visualizing Data 
